In [31]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_csv('podcastdata_dataset.csv')
df.head()


,id,guest,title,text
0,1,Max Tegmark,Life 3.0,"As part of MIT course 6S099, Artificial Genera..."
1,2,Christof Koch,Consciousness,As part of MIT course 6S099 on artificial gene...
2,3,Steven Pinker,AI in the Age of Reason,"You've studied the human mind, cognition, lang..."
3,4,Yoshua Bengio,Deep Learning,What difference between biological neural netw...
4,5,Vladimir Vapnik,Statistical Learning,The following is a conversation with Vladimir ...


In [33]:
def preprocess_tolower(text):
    text=text.lower()
    return text

df['text']=df['text'].apply(preprocess_tolower)

In [34]:
df.head()

,id,guest,title,text
0,1,Max Tegmark,Life 3.0,"as part of mit course 6s099, artificial genera..."
1,2,Christof Koch,Consciousness,as part of mit course 6s099 on artificial gene...
2,3,Steven Pinker,AI in the Age of Reason,"you've studied the human mind, cognition, lang..."
3,4,Yoshua Bengio,Deep Learning,what difference between biological neural netw...
4,5,Vladimir Vapnik,Statistical Learning,the following is a conversation with vladimir ...


In [35]:
passages=[]
for index,row in df.iterrows():
    transcript=row['text']
    passages.extend(transcript[i:i+1000] for i in range(0,len(transcript),1000))

passages_df=pd.DataFrame(passages,columns=['Passages'])
passages_df.to_csv('passages.tsv', sep='\t', index=False)


In [49]:
from datasets import Dataset, load_from_disk
import faiss

dataset = Dataset.from_dict({"text": passages})
def embed(texts):
    return np.random.random((len(texts), 768)).astype("float32")
dataset = dataset.map(lambda example: {"title": example["text"]})
dataset = dataset.map(lambda examples: {"embeddings": embed(examples["text"])}, batched=True)




Map: 100%|██████████| 38000/38000 [00:00<00:00, 101538.56 examples/s]


In [50]:
print(dataset)

Dataset({
    features: ['text', 'title', 'embeddings'],
    num_rows: 38000
})


In [51]:
dataset_path = "passages_dataset"

dataset.save_to_disk(dataset_path)

dataset.add_faiss_index(column="embeddings", index_name="embeddings")

index_path = "passages_index.faiss"

dataset.get_index("embeddings").save(index_path)

dataset = load_from_disk(dataset_path)

dataset.load_faiss_index(index_name="embeddings", file=index_path)

print("Dataset with FAISS index loaded successfully")

100%|██████████| 38/38 [00:00<00:00, 353.17it/s]


Dataset with FAISS index loaded successfully


In [52]:
print(dataset)

Dataset({
    features: ['text', 'title', 'embeddings'],
    num_rows: 38000
})


In [ ]:
import torch
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

tokenizer = RagTokenizer.from_pretrained('facebook/rag-sequence-nq')

In [53]:
retriever = RagRetriever.from_pretrained(
    'facebook/rag-sequence-nq',
    index_name="custom",
    indexed_dataset=load_from_disk(dataset_path)
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

ValueError: Missing faiss index in the dataset. Make sure you called `dataset.add_faiss_index` to compute it or `dataset.load_faiss_index` to load one from the disk.